# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city = pd.DataFrame(pd.read_csv("../WeatherPy/output_data/city.csv"))
city.head()


,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed
0,0,Huayucachi,84,PE,1659415498,80,-12.1333,-75.2333,10.28,0.70
1,1,Barentin,87,FR,1659415498,94,49.5453,0.9552,16.79,1.54
2,2,Bilibino,73,RU,1659415498,34,68.0546,166.4372,21.47,8.22
3,3,Laas,97,IT,1659415498,82,46.6166,10.7002,15.17,1.24
4,4,Mataura,10,NZ,1659415499,75,-46.1927,168.8643,6.93,5.11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
#Use the Lat and Lng as locations and Humidity as the weight.
locations = city[["Latitude", "Longitude"]]
humidity = city["Humidity"].astype(float)


In [4]:
#Add Heatmap layer to map.
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

#Create and add heat layer 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
plt.savefig("humiditymap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
purfect_weather=city.loc[(city['Temperature']>20) & (city['Temperature']<33) & (city['Wind Speed']<3) & (city['Cloudiness']<20) & (city['Humidity']<50)]
purfect_weather

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed
122,122,Anadyr,0,RU,1659415526,45,64.7500,177.4833,20.28,2.00
161,161,Saryozek,5,KZ,1659415535,28,44.3583,77.9753,27.63,1.16
194,194,Saint George,0,US,1659415418,40,37.1041,-113.5841,31.52,2.57
239,239,Gumdag,0,TM,1659415551,27,39.2061,54.5906,32.64,2.58
281,281,Sunnyside,0,US,1659415560,43,46.3237,-120.0087,27.25,2.06
282,282,Gilbués,16,BR,1659415560,44,-9.8317,-45.3439,20.79,2.65
334,334,Maracaju,0,BR,1659415572,41,-21.6144,-55.1683,21.07,1.32
431,431,Fethiye,0,TR,1659415594,42,36.6217,29.1164,28.93,0.67
441,441,Semnan,0,IR,1659415596,24,35.5729,53.3971,27.91,1.03
536,536,Igrim,18,RU,1659415619,44,63.1906,64.4162,20.89,1.62


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = purfect_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {"radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key}
    

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    Latitude = row["Latitude"]
    Longitude = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{Latitude},{Longitude}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 64.7349681,
                    "lng": 177.5160323
                },
                "viewport": {
                    "northeast": {
                        "lat": 64.73635547989272,
                        "lng": 177.5172848798927
                    },
                    "southwest": {
                        "lat": 64.73365582010727,
                        "lng": 177.5145852201073
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Chukotka hotel",
            "photos": [
                {
                    "height": 3000,
  

{
    "html_attributions": [],
    "next_page_token": "AeJbb3eM4m-e-bHfMnavWiZxqZFTUagzuP3vszHp3o0N5-mWcgc7s-f02Rbem6UvOPe4HC6ylosK-_DEF21UuRAPilKPq5bDTJEtk0ey2Y54YWAU7-S0ct914aRzSwJ3mhexiOMZH8qONtmXTg-nXcEc4SLYm7ZjlTaqou2bwNx_Km5V6vTeKk1B-nVSSDYOqDslLKrWmFLa0WvznQFyPTk3PnqUK3xbANTI12RvgYIr4fjXrS25kLJxrozn0bHWv5FAy3Hi905w6dNcIzlA8LezH7xa2aZFxCnB7uAjgpVC3HLj6hVAhjZodtVGMXvmeyiSMl0UqqJEqgcJxra741XTbONcasunPmGLAaiJDkkvMpYgWXWBhYLpNyG3DhRnbV1gIPnISc2yU7g8uWo7Eu9dcTJSWTB-Az6WudUK9tSh_WSsItU1H0urVkqQH4kJePco",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 36.8074038,
                    "lng": -114.1038088
                },
                "viewport": {
                    "northeast": {
                        "lat": 36.80908542989273,
                        "lng": -114.1028592701073
                    },
                    "southwest": {
                        "lat": 36.806385

{
    "html_attributions": [],
    "next_page_token": "AeJbb3doAMRzUus7U_XSwx4FRIiVaI0xmkaxKPFK5vuBwfmKftzxOCBQBDomUwlf_lXfv2gzqcOvWCJ3iRVUlJ_S2KysXymuqocsgI5tLnHN6dFFuj8N95MfZi78iqSs3aQ-e0RYs1I2JsMbv3Vc-fzfESvBhIGWujmNVsDx-G-uQJqOX6h8hYK1RMMZrqdinpTymQnVU3dIbe1Q5sdRUzQ8XLwf9D6LzMjplF4-gBwX0nefiqsxlU_8l6TT3OkkD0PhYIGWwZmXsxDRZslSdKToSAJLnIUQvm1IWsk-h_yFpSYUiV79yt-315-wfEzg_WIif9gDaS6sZwA09V1Fv_SqdIkqY4E7QfnI6jFm3KRsTj-DOf3gbLBOZbKnSI9Bje-NUdxLA8ydFk7RlNjm39Qo-UVLLfbsLcLWzcXvZkIj4bk04OT9jaymV9tCoZFPuZLx",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 46.21879250000001,
                    "lng": -119.7922276
                },
                "viewport": {
                    "northeast": {
                        "lat": 46.22018317989272,
                        "lng": -119.7908311201073
                    },
                    "southwest": {
                        "lat": 46

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.631868,
                    "lng": -55.156347
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.63059917010728,
                        "lng": -55.15498977010728
                    },
                    "southwest": {
                        "lat": -21.63329882989272,
                        "lng": -55.15768942989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Hotel Pousada da Serra",
            "opening_hours": {
                "open_now": true
     

{
    "html_attributions": [],
    "next_page_token": "AeJbb3eG8sQBzzrhadmKALVZSw-N8FGRr9uWg0UjtmRzgqPBx_Bx8ke5f5fpTYVr_vR1-JIEfQXBpQ0DOgPBsq0cFFq_7FEwGO5WvlcRmv1Xk8dvf5NgDfch9DNXoIJnIZjV8kCgaRbsFi2s5rtNH8LguQ5MpxshDThbGbCVJl85YdGplPh3LNm5ts1UIAAXu7EbOoeQHjJ4u2aFtAgUoRgtzuskE4cLwovdiYPXP0omtbwz5lZ2Y1oOXccR6jJhhD4qD4sDR89AziLTSq4NtyNZuTDrB1jHNv0wVetT8kb78XOwIIUCiq5yWw_wftCUdhzFiuE8x4Kw9SIrRw7rfSvY3GrDmYTpPikhpGmfLeBTX5RNmdBaGxOBPWSlJfYXmTKXrMBG9JYwnaP9p_vUabjOBdZNAHU9gEzH4Pzqty8occl_ZURT0xS4B8pMZPOD15v1",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 35.5699952,
                    "lng": 53.398537
                },
                "viewport": {
                    "northeast": {
                        "lat": 35.57143097989272,
                        "lng": 53.39990877989272
                    },
                    "southwest": {
                        "lat": 35.5687313201

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Temperature,Wind Speed,Hotel Name
0,122,Anadyr,0,RU,1659415526,45,64.7500,177.4833,20.28,2.00,Chukotka hotel
1,161,Saryozek,5,KZ,1659415535,28,44.3583,77.9753,27.63,1.16,ADLER
2,194,Saint George,0,US,1659415418,40,37.1041,-113.5841,31.52,2.57,"Holiday Inn Express & Suites Mesquite, an IHG ..."
3,239,Gumdag,0,TM,1659415551,27,39.2061,54.5906,32.64,2.58,Nebitchi Hotel
4,281,Sunnyside,0,US,1659415560,43,46.3237,-120.0087,27.25,2.06,Holiday Inn Express & Suites Prosser - Yakima ...
5,282,Gilbués,16,BR,1659415560,44,-9.8317,-45.3439,20.79,2.65,Hotel Camapuã
6,334,Maracaju,0,BR,1659415572,41,-21.6144,-55.1683,21.07,1.32,Hotel Pousada da Serra
7,431,Fethiye,0,TR,1659415594,42,36.6217,29.1164,28.93,0.67,Club & Hotel Letoonia
8,441,Semnan,0,IR,1659415596,24,35.5729,53.3971,27.91,1.03,Semnan Traditional Hotel
9,536,Igrim,18,RU,1659415619,44,63.1906,64.4162,20.89,1.62,Uyut


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info)
# Plot Heatmap
fig = gmaps.figure()
                            
# Add layer
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Display figure
plt.savefig("hotelsmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>